In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import datetime
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext
from tkinter import messagebox
from tkcalendar import Calendar

# Load the dataset
places = pd.read_csv('travel.csv')

# Preprocess the text data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(places['Place'])

# Create a DataFrame to keep track of the place names and their corresponding TF-IDF vectors
feature_names = vectorizer.get_feature_names_out()
places_df = pd.DataFrame(X.toarray(), columns=feature_names)
places_df['Place'] = places['Place']
places_df['City'] = places['City']
places_df['PLace_Ratings'] = places['PLace_Ratings']
places_df['City_Ratings'] = places['City_Ratings']
places_df['Best_time_to_visit'] = places['Best_time_to_visit']
places_df['Budget'] = places['Budget']

def recommend_places(city, start_date, end_date, budget):
    # Ensure budget column is converted to float
    places_df['Budget'] = places_df['Budget'].str.replace(',', '').str.extract(r'(\d+.\d+)').astype(float)

    # Filter the places DataFrame to only include places in the user's chosen city
    city_places = places_df[places_df['City'] == city]

    # Top 5 places with the highest place ratings
    top_5_places = city_places.sort_values('PLace_Ratings', ascending=False).head(5)

    # Top 5 alternate places within the user's budget
    budget_places = city_places[city_places['Budget'] <= budget].sort_values('PLace_Ratings', ascending=False).head(5)

    # Parse start and end dates
    start_date_obj = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date_obj = datetime.datetime.strptime(end_date, '%Y-%m-%d')

    # Top 5 places to visit during the user's chosen time of year
    best_time_places = city_places[city_places['Best_time_to_visit'].str.contains(start_date_obj.strftime('%B'))].sort_values('PLace_Ratings', ascending=False).head(5)

    # Concatenate the top 5 places, alternate places, and best time places DataFrames
    recommended_places = pd.concat([top_5_places, budget_places, best_time_places]).drop_duplicates()

    # If the user's input results in fewer than 5 recommended places, fill the remaining spots with random places
    if len(recommended_places) < 5:
        remaining_places = city_places.drop(recommended_places.index).sample(n=5-len(recommended_places))
        recommended_places = pd.concat([recommended_places, remaining_places])

    # Return the top 5 recommended places with their corresponding column names
    return recommended_places[['Place', 'City', 'PLace_Ratings', 'City_Ratings', 'Best_time_to_visit', 'Budget']]

def recommend_places_gui():
    def on_submit():
        city = city_entry.get()
        start_date = start_date_var.get()
        end_date = end_date_var.get()
        try:
            budget = float(budget_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid budget.")
            return

        recommendations = recommend_places(city, start_date, end_date, budget)

        result_text.config(state=tk.NORMAL)
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, recommendations.to_string(index=False))
        result_text.config(state=tk.DISABLED)

    def show_calendar(entry_var):
        def set_date():
            selected_date = cal.selection_get()
            entry_var.set(selected_date.strftime('%Y-%m-%d'))
            top.destroy()

        top = tk.Toplevel(window)
        cal = Calendar(top, font="Arial 14", selectmode='day', cursor="hand1")
        cal.pack(fill="both", expand=True)
        ttk.Button(top, text="OK", command=set_date).pack()


        top = tk.Toplevel(window)
        cal = Calendar(top, font="Arial 14", selectmode='day', cursor="hand1", year=datetime.date.today().year, month=datetime.date.today().month, day=datetime.date.today().day)
        cal.pack(fill="both", expand=True)
        ttk.Button(top, text="OK", command=set_date).pack()

    def clear_entries():
        city_entry.delete(0, tk.END)
        start_date_var.set('')
        end_date_var.set('')
        budget_entry.delete(0, tk.END)
        result_text.config(state=tk.NORMAL)
        result_text.delete(1.0, tk.END)
        result_text.config(state=tk.DISABLED)

    # Create main window
    window = tk.Tk()
    window.title("Travel Recommendations")

    # Create labels and entry widgets
    ttk.Label(window, text="City:").grid(row=0, column=0, padx=5, pady=5)
    city_entry = ttk.Entry(window)
    city_entry.grid(row=0, column=1, padx=5, pady=5)

    ttk.Label(window, text="Start Date:").grid(row=1, column=0, padx=5, pady=5)
    start_date_var = tk.StringVar()
    start_date_entry = ttk.Entry(window, textvariable=start_date_var, state='readonly')
    start_date_entry.grid(row=1, column=1, padx=5, pady=5)
    start_date_button = ttk.Button(window, text="Select", command=lambda: show_calendar(start_date_var))
    start_date_button.grid(row=1, column=2, padx=5, pady=5)

    ttk.Label(window, text="End Date:").grid(row=2, column=0, padx=5, pady=5)
    end_date_var = tk.StringVar()
    end_date_entry = ttk.Entry(window, textvariable=end_date_var, state='readonly')
    end_date_entry.grid(row=2, column=1, padx=5, pady=5)
    end_date_button = ttk.Button(window, text="Select", command=lambda: show_calendar(end_date_var))
    end_date_button.grid(row=2, column=2, padx=5, pady=5)

    ttk.Label(window, text="Budget:").grid(row=3, column=0, padx=5, pady=5)
    budget_entry = ttk.Entry(window)
    budget_entry.grid(row=3, column=1, padx=5, pady=5)

    # Create buttons
    submit_button = ttk.Button(window, text="Submit", command=on_submit)
    submit_button.grid(row=4, column=0, columnspan=3, pady=10)

    clear_button = ttk.Button(window, text="Clear", command=clear_entries)
    clear_button.grid(row=5, column=0, columnspan=3, pady=10)

    # Create text widget to display recommendations with a larger size
    result_text = scrolledtext.ScrolledText(window, height=15, width=100, wrap=tk.WORD, state=tk.DISABLED)
    result_text.grid(row=6, column=0, columnspan=3, padx=10, pady=10)

    # Start the main loop
    window.mainloop()

# Call the recommend_places_gui function
recommend_places_gui()



Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\91859\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\91859\AppData\Local\Temp\ipykernel_20504\4114669228.py", line 70, in on_submit
    recommendations = recommend_places(city, start_date, end_date, budget)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\91859\AppData\Local\Temp\ipykernel_20504\4114669228.py", line 30, in recommend_places
    places_df['Budget'] = places_df['Budget'].str.replace(',', '').str.extract(r'(\d+.\d+)').astype(float)
                          ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\91859\anaconda3\Lib\site-packages\pandas\core\generic.py", line 5989, in __getattr__
    return object.__getattribute__(self, name)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\91859\anaconda3\Lib\site-packages\pandas\core\accessor.py", line 224, in __get

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\91859\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\91859\AppData\Local\Temp\ipykernel_20504\4114669228.py", line 70, in on_submit
    recommendations = recommend_places(city, start_date, end_date, budget)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\91859\AppData\Local\Temp\ipykernel_20504\4114669228.py", line 30, in recommend_places
    places_df['Budget'] = places_df['Budget'].str.replace(',', '').str.extract(r'(\d+.\d+)').astype(float)
                          ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\91859\anaconda3\Lib\site-packages\pandas\core\generic.py", line 5989, in __getattr__
    return object.__getattribute__(self, name)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\91859\anaconda3\Lib\site-packages\pandas\core\accessor.py", line 224, in __get